In [10]:
#checked!

In [16]:
# add noise independently to each attribute
# use same epsilon and delta 
# use binary mechanism (https://diffprivlib.readthedocs.io/en/latest/modules/mechanisms.html#binary-mechanism) for sex
# use geometric mechanism (https://diffprivlib.readthedocs.io/en/latest/modules/mechanisms.html#geometric-mechanisms) for age
# use binary mechanism (https://diffprivlib.readthedocs.io/en/latest/modules/mechanisms.html#binary-mechanism) to select real value or representative random value for race

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from diffprivlib import tools as dp
from diffprivlib.mechanisms import Binary
from diffprivlib.mechanisms import Geometric
import pandas as pd
import random

#from all_us distribution - creates concentration of high occurence races in middle, low occurence on outside
real_order = [124, 116, 118, 123, 115, 117, 125, 51, 59, 111, 32, 102, 108, 42, 62, 37, 99, 44, 103, 48, 26, 96, 31, 106, 95, 45, 105, 38, 41, 107, 119, 97, 33, 43, 30, 94, 13, 79, 15, 83, 80, 27, 29, 12, 88, 76, 11, 78, 4, 17, 81, 75, 19, 74, 14, 6, 67, 68, 1, 10, 65, 5, 64, 63, 0, 66, 69, 71, 70, 2, 73, 8, 7, 84, 3, 72, 91, 77, 85, 20, 82, 21, 24, 9, 18, 28, 104, 22, 110, 87, 25, 86, 89, 92, 35, 90, 40, 36, 100, 50, 47, 16, 93, 39, 23, 98, 34, 56, 114, 57, 101, 113, 54, 46, 52, 49, 60, 55, 120, 53, 58, 109, 112, 61, 122, 121]

#races x 2 to account for hispanic and non hispanic; hispanic will be 0-62, nonhispanic 63-125
races = ['Total!!Population of one race!!White alone',
       'Total!!Population of one race!!Black or African American alone',
       'Total!!Population of one race!!American Indian and Alaska Native alone',
       'Total!!Population of one race!!Asian alone',
       'Total!!Population of one race!!Native Hawaiian and Other Pacific Islander alone',
       'Total!!Population of one race!!Some Other Race alone',
       'Total!!Two or More Races!!Population of two races!!White; Black or African American',
       'Total!!Two or More Races!!Population of two races!!White; American Indian and Alaska Native',
       'Total!!Two or More Races!!Population of two races!!White; Asian',
       'Total!!Two or More Races!!Population of two races!!White; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!White; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!Black or African American; American Indian and Alaska Native',
       'Total!!Two or More Races!!Population of two races!!Black or African American; Asian',
       'Total!!Two or More Races!!Population of two races!!Black or African American; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!Black or African American; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of two races!!American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!Asian; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; American Indian and Alaska Native',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; Asian',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of three races!!White; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!White; American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!White; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Black or African American; American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of three races!!Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!Black or African American; American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Black or African American; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!Black or African American; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Black or African American; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!White; American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!Black or African American; American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!Black or African American; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of six races!!White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
        'Total!!Population of one race!!White alone',
       'Total!!Population of one race!!Black or African American alone',
       'Total!!Population of one race!!American Indian and Alaska Native alone',
       'Total!!Population of one race!!Asian alone',
       'Total!!Population of one race!!Native Hawaiian and Other Pacific Islander alone',
       'Total!!Population of one race!!Some Other Race alone',
       'Total!!Two or More Races!!Population of two races!!White; Black or African American',
       'Total!!Two or More Races!!Population of two races!!White; American Indian and Alaska Native',
       'Total!!Two or More Races!!Population of two races!!White; Asian',
       'Total!!Two or More Races!!Population of two races!!White; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!White; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!Black or African American; American Indian and Alaska Native',
       'Total!!Two or More Races!!Population of two races!!Black or African American; Asian',
       'Total!!Two or More Races!!Population of two races!!Black or African American; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!Black or African American; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of two races!!American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!Asian; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; American Indian and Alaska Native',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; Asian',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of three races!!White; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!White; American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!White; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Black or African American; American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of three races!!Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!Black or African American; American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Black or African American; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!Black or African American; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Black or African American; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!White; American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!Black or African American; American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!Black or African American; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of six races!!White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race']



race_dict1 = {}
race_dict2 = {}

for i in range(len(races)):
    race_dict1[i] = races[i]
    race_dict2[races[i]] = i

tenuredict = {0: 'Own', 1:'Rent'}
tenuredict2 = {'Own': 0, 'Rent':1}

In [18]:
##########################################################################################################################
#CREATE BLOCK DATA
#input: csv containing block data 
#output: dataframe containing block data for age, sex, race, etc
##########################################################################################################################

def create_block_data(path):
    df = pd.read_csv(path)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

# df = create_block_data('../homemade_data/almeda.csv')
# orig_df = df.copy()

In [21]:
##########################################################################################################################
#MAKE DIFFERENTIALLY PRIVATE
#input: 1. dataframe dframe containing block data 2. race_data containing representative race vals 3. epsilon val (float)
#output: differentially private representation of dataframe df
##########################################################################################################################

def make_differentially_private(dft,  ep):
    cs = []
    ds = []
    es = []
    fs = []
    gs = []
    hs = []

    dframe = dft.copy()
    b_sex = Binary(epsilon=ep, value0="m", value1="f")
    #b_his = Binary(epsilon=ep, value0="0", value1="1")
    geo_age = Geometric(epsilon=ep, sensitivity=115)
#     b2 = Binary(epsilon=ep, value0="true value", value1="fake value") I don't think we use this -Miranda
    geo_race = Geometric(epsilon=ep, sensitivity=125)
    geo_size = Geometric(epsilon=ep, sensitivity=3)
    b_tenure = Binary(epsilon=ep, value0="Own", value1="Rent")
    for (columnName, columnData) in dframe.iteritems():
        if columnName == "sex":
            i = 0
            for c in columnData.values:          
                d = b_sex.randomise(c)
                dframe.at[i,'sex'] = d
                i +=1
                cs.append(c)
                ds.append(d)
                
        if columnName == "age":
            i = 0
            for e in columnData.values:
                f = geo_age.randomise(int(e))
                dframe.at[i,'age'] = f
                i +=1
                es.append(e)
                fs.append(f)
                
        if columnName == "race":
            i = 0
            for g in columnData.values:
                print(g)
                idx = real_order.index(race_dict2[g])
                val = geo_race.randomise(idx)%126
                h = race_dict1[real_order[val]]
                
                
                if(race_dict2.index(h) <= 62):
                    dframe.at[i, 'hispanic'] = 1
                    dframe.at[i, 'race'] = h
                else:
                    dframe.at[i, 'hispanic'] = 0
                    dframe.at[i, 'race'] = h
                i +=1
                
        if columnName == "household_size":
            i = 0
            for g in columnData.values:
                h = geo_size.randomise(g)%4 +1
                dframe.at[i, 'household_size'] = h
                i +=1
                
        if columnName == "household_tenure":
            i = 0
            for g in columnData.values:
#                 h = tenuredict[geo.randomise(tenuredict2[g])%2]
                h = b_tenure.randomise(g)
                dframe.at[i, 'household_tenure'] = h
                i +=1

    return dframe

        #testing




In [22]:
# Make dp_runs2 with 100 datasets per swap rate
# Naming convention: dprun_{epsilon}_{number from 0 to 100}.csv
counties = ['alameda', 'armstrong', 'cibola', 'fayette', 'grandforks', 'hawaii', 'jefferson', 'nantucket', 'washington']
#counties = ['armstrong', 'cibola', 'fayette', 'grandforks', 'hawaii', 'jefferson', 'nantucket', 'washington']


for county in counties:
    print(county)
    df_first = create_block_data('../homemade_data/'+county+'.csv')
    orig_df = df_first.copy()
    csv_orig_data = df_first.to_csv('dp_runs/'+county+'/orig_data.csv', index = True)

    epsilon = np.arange(.01, 7, .1, float)

    for e in epsilon:
        print(str(e)[0:4])
        for i in range(100):
            df = df_first.copy()
            dframe = make_differentially_private(df, e)
            filename = "dp_runs/"+county+"/dprun_"+str(e)[0:4]+"_"+str(i)+".csv"
            csv_orig_data = dframe.to_csv(filename, index = True)

alameda
0.01
Total!!Population of one race!!White alone


AttributeError: 'dict' object has no attribute 'index'